In [24]:
import pandas as pd
data1 = pd.read_excel('C:/Users/HP/Desktop/SentiScores_NSC.xlsx', header = 0)
data2 = pd.read_excel('C:/Users/HP/Desktop/Athar_Author_new.xlsx', header = 0)
data2.shape

(3069, 15)

In [25]:
print(data1.head())
print(data2.head())
data1 = data1[['Paper', 'No of citing paper', 'Total Citations', 'Total Score']]

      Paper  No of citing paper  Total Citations  Total Score
0  A00-2024                  13               23        4.125
1  A92-1018                  60               81        3.750
2  C02-1007                   6               16        3.750
3  C04-1162                   5                7        0.500
4  C08-1005                   2                2        0.625
      Paper          Author1                Author2          Author3 Author4  \
0  A00-1004      Chen, Jiang          Nie, Jian-Yun              NaN     NaN   
1  A00-1005      Bagga, Amit    Strzalkowski, Tomek  Wise, G. Bowden     NaN   
2  A00-1007    Jonsson, Arne         Dahlback, Nils              NaN     NaN   
3  A00-1011   Aone, Chinatsu  Ramos-Santacruz, Mila              NaN     NaN   
4  A00-1012  Stevenson, Mark     Gaizauskas, Robert              NaN     NaN   

  Author5 Author6 Author7 Author8 Author9 Author10 Author11 Author12 Author13  \
0     NaN     NaN     NaN     NaN     NaN      NaN      NaN      N

In [27]:
# Melting the data similar to "Author_SelfCitation_And_EdgeList.ipynb" 
data2_long = data2.melt(id_vars=['Paper'], value_vars=data2.columns[1:], var_name='AuthorNumber', value_name='Author')

""" Merge data1 and data2_long on 'Paper Id' to get the Citation count 
and Total Sentiment score alongside the article-author pair"""

result = pd.merge(data1, data2_long, on='Paper')

# Print the result
print(result)

         Paper  No of citing paper  Total Citations  Total Score AuthorNumber  \
0     A00-2024                  13               23        4.125      Author1   
1     A00-2024                  13               23        4.125      Author2   
2     A00-2024                  13               23        4.125      Author3   
3     A00-2024                  13               23        4.125      Author4   
4     A00-2024                  13               23        4.125      Author5   
...        ...                 ...              ...          ...          ...   
2501  W96-0213                 166              244       10.500     Author10   
2502  W96-0213                 166              244       10.500     Author11   
2503  W96-0213                 166              244       10.500     Author12   
2504  W96-0213                 166              244       10.500     Author13   
2505  W96-0213                 166              244       10.500     Author14   

                 Author  
0

In [28]:
# Removing NULL data
result.fillna('', inplace=True)
filtered_data = result[result['Author']!= '']

In [29]:
# This will contain the total no of citations and sentiment scores for each article of every author--- useful for various metrics
filtered_data.head()

,Paper,No of citing paper,Total Citations,Total Score,AuthorNumber,Author
0,A00-2024,13,23,4.125,Author1,"Jing, Hongyan"
1,A00-2024,13,23,4.125,Author2,"McKeown, Kathleen"
14,A92-1018,60,81,3.750,Author1,"Cutting, Doug"
15,A92-1018,60,81,3.750,Author2,"Kupiec, Julian"
16,A92-1018,60,81,3.750,Author3,"Pedersen, Jan"


In [30]:
# Creating a unique list of Authors
authors = []
for i in filtered_data.index:
    if(filtered_data['Author'][i] not in authors):
        authors.append(filtered_data['Author'][i])

In [31]:
ncp = []
tc = []
tsc = []
# Creating lists for Total Sentiment Score, Total citations and total no of citing papers for each author
for i in authors:
    cp = 0
    tot = 0
    sc = 0
    for j in filtered_data.index:
        if(filtered_data['Author'][j] == i):
            cp += filtered_data['No of citing paper'][j]
            tot += filtered_data['Total Citations'][j]
            sc += filtered_data['Total Score'][j]
    ncp.append(cp)
    tc.append(tot)
    tsc.append(sc)

In [32]:
print(len(authors), len(ncp), len(tc), len(tsc))

307 307 307 307


In [33]:
# Storing the score values of Author in Dataframe
df1 = pd.DataFrame()
df1['Author'] = authors
df1['No of Citing Papers'] = ncp
df1['No of Citation Sentences'] = tc
df1['Sentiment Score'] = tsc

In [34]:
df1.to_excel('C:/Users/HP/Desktop/Author_scores.xlsx', index = False)

In [35]:
filtered_data.drop('AuthorNumber', axis = 1)

,Paper,No of citing paper,Total Citations,Total Score,Author
0,A00-2024,13,23,4.125,"Jing, Hongyan"
1,A00-2024,13,23,4.125,"McKeown, Kathleen"
14,A92-1018,60,81,3.750,"Cutting, Doug"
15,A92-1018,60,81,3.750,"Kupiec, Julian"
16,A92-1018,60,81,3.750,"Pedersen, Jan"
...,...,...,...,...,...
2451,W91-0208,5,6,0.875,"Atkins, B. T. S."
2464,W93-0113,4,6,-0.875,"Grefenstette, Gregory"
2478,W95-0107,109,210,4.875,"Ramshaw, Lance"
2479,W95-0107,109,210,4.875,"Marcus, Mitch"


In [36]:
# Computing H-Index
h_index = []
for j in authors:
    newdata = filtered_data[filtered_data['Author'] == j]
    newdata = newdata.sort_values(by=['No of citing paper'], ascending=False)
    i = 0
    while i < len(newdata) and i < newdata['No of citing paper'].iloc[i]:
        i += 1
    h_index.append(i)


In [37]:
len(h_index)

307

In [ ]:
for i in range (357):
    print(authors[i], h_index[i])

In [39]:
filtered_data[filtered_data['Author'] == 'McDonald, Ryan']

,Paper,No of citing paper,Total Citations,Total Score,AuthorNumber,Author
280,D07-1013,13,21,-3.500,Author1,"McDonald, Ryan"
1596,P07-1055,2,5,2.375,Author1,"McDonald, Ryan"
1765,P08-1036,5,6,0.625,Author2,"McDonald, Ryan"
2297,W06-1615,14,48,4.625,Author2,"McDonald, Ryan"


In [40]:
# Sentiment Based H index
s_index = []
for j in authors:
    newdata = filtered_data[filtered_data['Author'] == j]
    newdata = newdata.sort_values(by=['Total Score'], ascending=False)
    i = 0
    while i < len(newdata) and i < newdata['Total Score'].iloc[i]:
        i += 1
    s_index.append(i)

In [ ]:
for i in range (357):
    print(authors[i], s_index[i])

In [42]:
filtered_data.to_excel('C:/Users/HP/Desktop/Author_Papers_Athar.xlsx', index = False)

In [43]:
# This DF will contain the H-index and S-index
indexes = pd.DataFrame()
indexes['Author'] = authors
indexes['h index'] = h_index
indexes['s index'] = s_index
indexes.to_excel('C:/Users/HP/Desktop/Author_indexes_athar.xlsx', index = False)